In [1]:
import re

In [2]:
sequence_length = 16

In [3]:
def txt_to_list(path, sequence_length):
    with open(path, 'r') as f:
        txt = str(f.read())
    txt_list = txt.split('\n')
    text_label_list = list()
    tmp = list()
    for line in txt_list:
        if line == '--------------------':
            text_label_list.append(tmp)
            tmp = list()
            continue
        if line == '':
            continue
        tmp.append(line)
    x = list()
    y = list()
    for text_label in text_label_list:
        text = text_label[0]
        text_list = [i for i in text]
        label = text_label[2:]
        label_list = ['O' for i in range(len(text))]
        for i in label:
            entity = i.split('\t')
            if int(entity[1]) > int(entity[2]):
                continue
            b = int(entity[1])
            label_list[b] = 'B-{}'.format(entity[-1])
            for j in range(int(entity[1])+1, int(entity[2])):
                label_list[j] = 'I-{}'.format(entity[-1])
                
        while len(text_list) > sequence_length:            
            x.append(text_list[:sequence_length])
            y.append(label_list[:sequence_length])
            text_list = text_list[sequence_length:]
            label_list = label_list[sequence_length:]
        x.append(text_list)
        y.append(label_list)
        
    return [x, y]

In [4]:
x1, y1 = txt_to_list('../datasets/stage1/SampleData_deid.txt', sequence_length)

In [5]:
x2, y2 = txt_to_list('../datasets/stage2/train_1_update.txt', sequence_length)

In [6]:
x4, y4 = txt_to_list('../datasets/stage4/train_2.txt', sequence_length)

In [7]:
x = x1 + x2 + x4
y = y1 + y2 + y4
print(len(x))
print(len(y))

44900
44900


In [8]:
# print(x[0])

In [9]:
# print(y[0])

In [10]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
print(len(train_x), len(train_y))
print(len(valid_x), len(valid_y))
print(len(test_x), len(test_y))

28736 28736
7184 7184
8980 8980


In [11]:
import kashgari
from kashgari.tasks.labeling.models import BiLSTM_CRF_Model
from kashgari.embeddings import BERTEmbedding
kashgari.config.use_cudnn_cell = True

bert_embed = BERTEmbedding('chinese_L-12_H-768_A-12',
                           task=kashgari.LABELING,
                           sequence_length=sequence_length,
                          )

model = BiLSTM_CRF_Model(bert_embed)
model = BiLSTM_CRF_Model()
model.fit(train_x,
          train_y,
          valid_x,
          valid_y,
          epochs=20,
         )



╭─────────────────────────────────────────────────────────────────────────╮
│ ◎ ○ ○ ░░░░░░░░░░░░░░░░░░░░░  Important Message  ░░░░░░░░░░░░░░░░░░░░░░░░│
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│              We renamed again for consistency and clarity.              │
│                   From now on, it is all `kashgari`.                    │
│  Changelog: https://github.com/BrikerMan/Kashgari/releases/tag/v1.0.0   │
│                                                                         │
│         | Backend          | pypi version   | desc           |          │
│         | ---------------- | -------------- | -------------- |          │
│         | TensorFlow 2.x   | kashgari 2.x.x | coming soon    |          │
│         | TensorFlow 1.14+ | kashgari 1.x.x |                |          │
│         | Keras            | kashgari 0.x.x | legacy version |          │
│           

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 16)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 16, 100)           174900    
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 16, 256)           234496    
_________________________________________________________________
layer_dense (Dense)          (None, 16, 64)            16448     
_________________________________________________________________
layer_crf_dense (Dense)      (None, 16, 28)            1820      
_________________________________________________________________
layer_crf (CRF)              (None, 16, 28)            784       
Total params: 428,448
Trainable params: 428,448
Non-trainable params: 0
_____________________________________________________

In [ ]:
model.evaluate(test_x, test_y)
model.save('model/BiLSTM_CRF_Model_len16_epoch20')

In [12]:
model = kashgari.utils.load_model('model/BiLSTM_CRF_Model_len16_epoch20')

/home/Danny/anaconda3/lib/python3.6/site-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/home/Danny/anaconda3/lib/python3.6/site-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/home/Danny/anaconda3/lib/python3.6/site-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/home/Danny/anaconda3/lib/python3.6/site-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


In [13]:
import pandas as pd
df = pd.read_csv('../datasets/stage4/development_2.csv')

In [15]:
def predict_NER(model, text, sequence_length):
    x_list = list()
    while len(text) > sequence_length:
        x_list.append([i for i in text[:sequence_length]])
        text = text[sequence_length:]        
    x_list.append([i for i in text[:sequence_length]])
    y_list = model.predict(x_list)
    merge_list = list()
    for i in y_list:
        for j in i:
            merge_list.append(j)
    return merge_list   

In [16]:
def output_NER(article_id, text, y):
    output = str()
    flag = False
    for i, j in enumerate(y):
        if j != 'O':
            if j[0] == 'B':
                start_position = i
                entity_type = j.split('-')[-1]
                flag = True
        elif j == 'O' and flag == True:
            end_position = i
            flag = False
            entity_text = text[start_position: end_position]
            output += '{}\t{}\t{}\t{}\t{}\n'.format(article_id, start_position, end_position, entity_text, entity_type)  
    return output

In [17]:
output="article_id\tstart_position\tend_position\tentity_text\tentity_type\n"
for article_id, text in zip(df['article_id'], df['text']):
    y = predict_NER(model, text, sequence_length)
    output += output_NER(article_id, text, y)    

In [18]:
print(output)

article_id	start_position	end_position	entity_text	entity_type
0	326	328	25	med_exam
0	370	372	晚上	time
0	409	411	4點	med_exam
0	423	427	來一小時	time
0	615	619	11點多	time
0	644	647	這兩天	time
0	817	819	晚上	time
0	1168	1172	7月21	time
0	1571	1574	10多	med_exam
0	1679	1680	兩	time
0	2200	2203	13號	time
0	2205	2208	第一次	time
0	2226	2228	3天	time
0	2319	2322	禮拜一	time
0	2332	2335	禮拜一	time
0	2372	2375	前幾天	time
0	2415	2416	水	name
0	2449	2451	兩天	time
0	2624	2628	下下禮拜	time
0	2736	2740	魯克林區	location
0	2780	2782	北區	location
0	2807	2810	下禮拜	time
0	2859	2862	1小時	time
0	3066	3070	那下禮拜	time
0	3340	3343	下禮拜	time
1	59	62	108	med_exam
1	66	69	108	med_exam
1	457	460	五月中	time
1	671	674	三個月	time
2	27	32	8月17號	time
2	40	45	上上個禮拜	time
2	53	56	110	med_exam
2	111	112	6	med_exam
2	253	256	126	ID
2	279	282	2年前	time
2	492	494	外國	location
2	496	498	泰國	name
2	524	526	釣蝦	name
2	718	720	斜斜	location
2	770	772	斜斜	name
2	779	781	斜斜	location
2	980	982	兩天	time
2	1087	1088	下	time
2	1114	1116	半年	time
2	1118	1120	半年	time
2	1139	1141	爸爸	fam

In [19]:
output_path='output/BiLSTM_CRF_Model_len16_epoch20.tsv'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(output)